In [2]:
import gmaps
from config import gkey2
from config import gkey
import requests
import json

In [3]:
import pandas as pd
import numpy as np

In [4]:
infile = pd.read_csv("Austin_driving_distance")
df = pd.DataFrame(infile)

In [12]:
df.tail(3)

,School,Closest Store,Store Address,Distance in Miles,Percent in Poverty,School Lat,School Long,Store Lat,Store Long
58,Overton Elementary School,H-E-B,"7112 Ed Bluestein Blvd #125, Austin",1.87,94.68,30.300187,-97.636701,30.312068,-97.664756
59,Perez Elementary School,H-E-B,"6607 S IH 35 Frontage Rd, Austin",1.46,88.01,30.170521,-97.756186,30.188724,-97.768718
60,Webb Primary School,Walmart Supercenter,"1030 Norwood Park Boulevard, Austin",1.15,97.48,30.332413,-97.707407,30.339045,-97.689688


In [6]:
# Create arrays of school and store coordinates so I can call by index in the loop

school_lat = df["School Lat"]
school_long = df["School Long"]
store_lat = df["Store Lat"]
store_long = df["Store Long"]
store_address = df["Store Address"]

In [7]:
# I need to know how many schools are in this dataframe so I can run a later loop
schools = len(store_address)
schools

61

In [8]:
base_url = "https://maps.googleapis.com/maps/api/distancematrix/json?"

In [21]:
# Getting the distance, driving-wise, between school and store and getting address
# as well to make sure everything matches up

i = 0
dest_address = []
orig_address = []
driving_distance = []
duration = []

while i < schools:
    
    params = {
    "origins": f"{school_lat[i]},{school_long[i]}", 
    "key": gkey2, 
    "destinations": f"{store_lat[i]},{store_long[i]}", 
    "units": "imperial"}
    
    data = requests.get(base_url, params = params).json()
    
    dest_address.append(data["destination_addresses"][0])
    orig_address.append(data["origin_addresses"][0])
    driving_distance.append(data["rows"][0]["elements"][0]["distance"]["text"])
    
        
    i += 1


In [24]:
# print(json.dumps(data, indent=4, sort_keys=True))

In [25]:
# print(orig_address,dest_address, driving_distance)

In [27]:
temp = pd.DataFrame()
temp["School"] = df["School"]
temp["Store"] = df["Closest Store"]
temp["Driving Distance"] = driving_distance

In [28]:
temp.head()

,School,Store,Driving Distance
0,Allison Elementary,H-E-B,1.0 mi
1,Andrews Elementary School,H-E-B,1.2 mi
2,Becker Elementary School,H-E-B,1.3 mi
3,Blackshear Elementary School,H-E-B,1.2 mi
4,Blanton Elementary,H-E-B,1.0 mi


In [43]:
# Getting the distance, using the bus, between school and store and getting address
# as well to make sure everything matches up

i = 0
dest_address2 = []
orig_address2 = []
bus_distance = []
bus_fare = []

while i < schools:
    
    params = {
    "origins": f"{school_lat[i]},{school_long[i]}", 
    "key": gkey2, 
    "destinations": f"{store_lat[i]},{store_long[i]}", 
    "units": "imperial",
    "mode": "transit",
    "transit_mode": "bus"}
    
    data2 = requests.get(base_url, params = params).json()
    
    dest_address2.append(data2["destination_addresses"][0])
    orig_address2.append(data2["origin_addresses"][0])
    bus_distance.append(data2["rows"][0]["elements"][0]["distance"]["text"])
#     bus_fare.append(data2["rows"][0]["elements"][0]["fare"]["text"])
    
        
    i += 1


In [ ]:
# print(json.dumps(data2, indent=4, sort_keys=True))

In [48]:
# print(bus_distance)

In [45]:
temp["Bus"] = bus_distance

In [49]:
# Making a dataframe of info we want to keep

transport_df = pd.DataFrame()

In [54]:
transport_df["School"] = df["School"]
# transport_df["School Address"]
transport_df["Closest Store"] = df["Closest Store"]
transport_df["Store Address"] = df["Store Address"]
transport_df["Driving Distance (miles)"] = driving_distance
transport_df["Bus Distance (miles)"] = bus_distance
transport_df["Percent in Poverty"] = df["Percent in Poverty"]
transport_df["Location"] = "Austin"

In [55]:
transport_df.head()

,School,Closest Store,Store Address,Driving Distance (miles),Bus Distance (miles),Percent in Poverty,Location
0,Allison Elementary,H-E-B,"2110 W Slaughter Ln, Austin",1.0 mi,1.3 mi,92.00,Austin
1,Andrews Elementary School,H-E-B,"7112 Ed Bluestein Blvd #125, Austin",1.2 mi,1.6 mi,91.02,Austin
2,Becker Elementary School,H-E-B,"2400 S Congress Ave, Austin",1.3 mi,1.3 mi,64.52,Austin
3,Blackshear Elementary School,H-E-B,"2701 E 7th St, Austin",1.2 mi,1.2 mi,82.49,Austin
4,Blanton Elementary,H-E-B,"1801 E 51st St, Austin",1.0 mi,1.1 mi,65.23,Austin


In [56]:
transport_df.to_csv("Austin_Distances.csv", index = False)